In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.\
        builder.\
        appName("pyspark-hello-world").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

23/02/11 14:48:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# read the parquet files
df = spark.read.parquet("./data/fhv_tripdata_2020-01.parquet")

In [3]:
df.head(5)

[Row(dispatching_base_num='B00001', pickup_datetime=datetime.datetime(2020, 1, 1, 0, 30), dropOff_datetime=datetime.datetime(2020, 1, 1, 1, 44), PUlocationID=264.0, SR_Flag=None, Affiliated_base_number='B00001'),
 Row(dispatching_base_num='B00001', pickup_datetime=datetime.datetime(2020, 1, 1, 0, 30), dropOff_datetime=datetime.datetime(2020, 1, 1, 0, 47), PUlocationID=264.0, SR_Flag=None, Affiliated_base_number='B00001'),
 Row(dispatching_base_num='B00009', pickup_datetime=datetime.datetime(2020, 1, 1, 0, 48), dropOff_datetime=datetime.datetime(2020, 1, 1, 1, 19), PUlocationID=264.0, SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00009', pickup_datetime=datetime.datetime(2020, 1, 1, 0, 34), dropOff_datetime=datetime.datetime(2020, 1, 1, 0, 43), PUlocationID=264.0, SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00009', pickup_datetime=datetime.datetime(2020, 1, 1, 0, 23), dropOff_datetime=datetime.datetime(2020, 1, 1, 0, 32), PUl

In [10]:
df.schema

StructType(List(StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,TimestampType,true),StructField(dropOff_datetime,TimestampType,true),StructField(PUlocationID,DoubleType,true),StructField(SR_Flag,IntegerType,true),StructField(Affiliated_base_number,StringType,true)))

In [23]:
# changing the schema
# PUlocationID change from DoubleType to IntegerType for better storage efficiency)
# integer: 4 bytes, long: 8 bytes
from pyspark.sql import types

In [ ]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropOff_datetime', types.TimestampType(), True),
    types.StructField('PUlocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [ ]:
# change the schema of the parquet file
# have decodeToInt error
# df_edit = spark.read.schema(schema).parquet("./data/fhv_tripdata_2020-01.parquet")

In [ ]:
# split the files into partitions, so that one file can be processed by multiple executors
no_of_partitions = 24
df.repartition(no_of_partitions)

In [ ]:
# output the partitioned file
df.write.parquet('fhvhv/2020/01')

In [13]:
df.select('pickup_datetime', 'dropOff_datetime', 'PUlocationID')\
    .filter(df.dispatching_base_num == 'B00001')\
    .show()

+-------------------+-------------------+------------+
|    pickup_datetime|   dropOff_datetime|PUlocationID|
+-------------------+-------------------+------------+
|2020-01-01 00:30:00|2020-01-01 01:44:00|       264.0|
|2020-01-01 00:30:00|2020-01-01 00:47:00|       264.0|
|2020-01-01 04:00:00|2020-01-01 04:40:00|       264.0|
|2020-01-01 05:30:00|2020-01-01 06:11:00|       264.0|
|2020-01-01 10:45:00|2020-01-01 11:12:00|       264.0|
|2020-01-01 11:47:00|2020-01-01 12:42:00|       264.0|
|2020-01-01 11:14:00|2020-01-01 12:22:00|       264.0|
|2020-01-01 13:00:00|2020-01-01 19:46:00|       264.0|
|2020-01-01 13:45:00|2020-01-01 14:45:00|       264.0|
|2020-01-01 14:28:00|2020-01-01 15:29:00|       264.0|
|2020-01-01 15:00:00|2020-01-01 16:15:00|       264.0|
|2020-01-01 16:12:00|2020-01-01 18:04:00|       264.0|
|2020-01-01 18:15:00|2020-01-01 18:28:00|       264.0|
|2020-01-01 19:23:00|2020-01-01 21:05:00|       264.0|
|2020-01-01 20:45:00|2020-01-01 22:11:00|       264.0|
|2020-01-0

23/02/11 15:14:16 WARN HeartbeatReceiver: Removing executor 1 with no recent heartbeats: 303151 ms exceeds timeout 120000 ms
23/02/11 15:14:16 WARN HeartbeatReceiver: Removing executor 0 with no recent heartbeats: 308102 ms exceeds timeout 120000 ms
23/02/11 15:14:16 ERROR TaskSchedulerImpl: Lost executor 1 on 172.25.0.5: Executor heartbeat timed out after 303151 ms
23/02/11 15:14:16 ERROR TaskSchedulerImpl: Lost executor 0 on 172.25.0.4: Executor heartbeat timed out after 308102 ms
23/02/11 15:26:42 WARN HeartbeatReceiver: Removing executor 2 with no recent heartbeats: 305528 ms exceeds timeout 120000 ms
23/02/11 15:26:42 ERROR TaskSchedulerImpl: Lost executor 2 on 172.25.0.5: Executor heartbeat timed out after 305528 ms
23/02/11 22:43:34 WARN HeartbeatReceiver: Removing executor 3 with no recent heartbeats: 301142 ms exceeds timeout 120000 ms
23/02/11 22:43:35 ERROR TaskSchedulerImpl: Lost executor 3 on 172.25.0.4: Executor heartbeat timed out after 301142 ms
23/02/11 22:53:41 WARN H

In [21]:
# user defined functions
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if (num % 7 == 0):
        # return number in hex
        return f's/{num:03x}'
    else:
        return f's/{num:03x}'
    
crazy_stuff('B00001')

's/001'

In [24]:
# save user defined functions into pyspark for future usage
crazy_stuff_udf = f.udf(crazy_stuff, returnType = types.StringType())

In [25]:
from pyspark.sql import functions as f

# add new date columns, which shows only the date information
# add new column that uses udf
df\
    .withColumn('pickup_date', f.to_date(df.pickup_datetime))\
    .withColumn('dropoff_date', f.to_date(df.dropOff_datetime))\
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num))\
    .select('base_id', 'pickup_date', 'dropOff_date', 'PUlocationID')\
    .show()

+-------+-----------+------------+------------+
|base_id|pickup_date|dropOff_date|PUlocationID|
+-------+-----------+------------+------------+
|  s/001| 2020-01-01|  2020-01-01|       264.0|
|  s/001| 2020-01-01|  2020-01-01|       264.0|
|  s/009| 2020-01-01|  2020-01-01|       264.0|
|  s/009| 2020-01-01|  2020-01-01|       264.0|
|  s/009| 2020-01-01|  2020-01-01|       264.0|
|  s/009| 2020-01-01|  2020-01-01|       264.0|
|  s/00d| 2020-01-01|  2020-01-01|       264.0|
|  s/00d| 2020-01-01|  2020-01-01|       264.0|
|  s/00d| 2020-01-01|  2020-01-01|       264.0|
|  s/00e| 2020-01-01|  2020-01-01|       264.0|
|  s/01f| 2020-01-01|  2020-01-01|       264.0|
|  s/025| 2020-01-01|  2020-01-01|       264.0|
|  s/025| 2020-01-01|  2020-01-01|       264.0|
|  s/025| 2020-01-01|  2020-01-01|       264.0|
|  s/025| 2020-01-01|  2020-01-01|       264.0|
|  s/025| 2020-01-01|  2020-01-01|       264.0|
|  s/025| 2020-01-01|  2020-01-01|       264.0|
|  s/025| 2020-01-01|  2020-01-01|      

23/02/12 07:14:13 ERROR TaskSchedulerImpl: Lost executor 86 on 172.25.0.4: worker lost
23/02/12 07:14:13 ERROR TaskSchedulerImpl: Lost executor 85 on 172.25.0.5: worker lost
23/02/12 07:20:11 ERROR TaskSchedulerImpl: Lost executor 87 on 172.25.0.4: worker lost
23/02/12 07:20:11 ERROR TaskSchedulerImpl: Lost executor 88 on 172.25.0.5: worker lost
23/02/12 07:26:08 ERROR TaskSchedulerImpl: Lost executor 90 on 172.25.0.5: worker lost
23/02/12 07:26:08 ERROR TaskSchedulerImpl: Lost executor 89 on 172.25.0.4: worker lost
23/02/12 07:44:13 ERROR TaskSchedulerImpl: Lost executor 91 on 172.25.0.5: worker lost
23/02/12 07:44:13 ERROR TaskSchedulerImpl: Lost executor 92 on 172.25.0.4: worker lost


Spark allows users to use both SQL and a programming language to process the data according to the various business requirements.